<a href="https://colab.research.google.com/github/Saputoa21/Machine-Translation/blob/main/filterMT_AdvancedMT_2025W.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://github.com/ymoslem/Adaptive-MT-LLM-Fine-tuning/blob/main/Data-Processing-Adaptive-MT.ipynb
#https://github.com/ymoslem/MT-Preparation/tree/main

In [2]:
!git clone https://github.com/ymoslem/MT-Preparation.git

Cloning into 'MT-Preparation'...
remote: Enumerating objects: 323, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 323 (delta 35), reused 21 (delta 20), pack-reused 268 (from 2)
Receiving objects: 100% (323/323), 94.95 KiB | 1.38 MiB/s, done.
Resolving deltas: 100% (156/156), done.


In [3]:
%cd MT-Preparation

/content/MT-Preparation


In [4]:
!pip3 install --user -r requirements.txt

In [5]:
%cd ..

/content


In [6]:
!ls

MT-Preparation	sample_data


In [7]:
!python3 MT-Preparation/filtering/filter.py train.en-de.en train.en-de.de en de

Dataframe shape (rows, columns): (50000, 2)
--- Rows with Empty Cells Deleted	--> Rows: 49997
--- Duplicates Deleted			--> Rows: 49997
--- Source-Copied Rows Deleted		--> Rows: 49997
--- Too Long Source/Target Deleted	--> Rows: 48399
--- Too Short Source/Target Deleted	--> Rows: 48321
--- HTML Removed			--> Rows: 48321
--- Rows will remain true-cased		--> Rows: 48321
--- Rows with Empty Cells Deleted	--> Rows: 48315
--- Rows Shuffled			--> Rows: 48315
--- Source Saved: train.en-de.en-filtered.en
--- Target Saved: train.en-de.de-filtered.de


In [8]:
!python3 MT-Preparation/filtering/semantic_filter.py train.en-de.en-filtered.en train.en-de.de-filtered.de en de 10000 0.45

2025-10-28 09:29:05.171515: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761643745.191420     893 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761643745.197664     893 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761643745.212276     893 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761643745.212301     893 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761643745.212305     893 computation_placer.cc:177] computation placer alr

In [9]:
!wc -l *

wc: model_cache: Is a directory
       0 model_cache
wc: MT-Preparation: Is a directory
       0 MT-Preparation
wc: sample_data: Is a directory
       0 sample_data
   50000 train.en-de.de
   48315 train.en-de.de-filtered.de
   47614 train.en-de.de-filtered.de.semantic.de
   50000 train.en-de.en
   48315 train.en-de.en-filtered.en
   47614 train.en-de.en-filtered.en.semantic.en
  291858 total


# **My corpus text**

I have chosen MultiUN (v1) corpus with the language pair EN-RU and size of 11,654,416 sentences. I splitted it into train (80k), validation (10k) and test (10k) sets.

The created files are:

*   UN-train.en-ru.en
*   UN-train.en-ru.ru
*   UN-test.en-ru.en
*   UN-test.en-ru.ru
*   UN-val.en-ru.en
*   UN-val.en-ru.ru

Link: https://opus.nlpl.eu/MultiUN/en&ru/v1/MultiUN


## Corpus splitting

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
file_ru = "/content/drive/MyDrive/Colab Notebooks/ru.txt.gz"
file_en = "/content/drive/MyDrive/Colab Notebooks/en.txt.gz"

In [12]:
import gzip

def extract_first_n_lines(gz_path, n_lines=100000):
    lines = []
    with gzip.open(gz_path, 'rt', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= n_lines:
                break
            lines.append(line)
    print(f"Extracted {len(lines):,} lines from {gz_path}")
    return lines

In [13]:
lines_ru = extract_first_n_lines(file_ru, n_lines=100000)
lines_en = extract_first_n_lines(file_en, n_lines=100000)

Extracted 100,000 lines from /content/drive/MyDrive/Colab Notebooks/ru.txt.gz
Extracted 100,000 lines from /content/drive/MyDrive/Colab Notebooks/en.txt.gz


In [14]:
import random

def split_data(lines, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    n = len(lines)
    n_train = int(n * train_ratio)
    n_val = int(n * val_ratio)

    train = lines[:n_train]
    val = lines[n_train:n_train + n_val]
    test = lines[n_train + n_val:]

    return train, val, test

In [15]:
train_ru, val_ru, test_ru = split_data(lines_ru)
train_en, val_en, test_en = split_data(lines_en)

print(len(train_ru))
print(len(val_ru))
print(len(test_ru))

80000
10000
10000


In [16]:
def save_bilingual_splits(prefix="UN", src_lang="en", tgt_lang="ru",
                          train_src=None, train_tgt=None,
                          val_src=None, val_tgt=None,
                          test_src=None, test_tgt=None):

    def save(name, src_data, tgt_data):
        src_file = f"{prefix}.{src_lang}-{tgt_lang}.{name}.{src_lang}"
        tgt_file = f"{prefix}.{src_lang}-{tgt_lang}.{name}.{tgt_lang}"

        with open(src_file, "w", encoding="utf-8") as f_src:
            f_src.writelines(src_data)
        with open(tgt_file, "w", encoding="utf-8") as f_tgt:
            f_tgt.writelines(tgt_data)

        print(f"Saved {src_file} ({len(src_data):,} lines)")
        print(f"Saved {tgt_file} ({len(tgt_data):,} lines)")

    if train_src and train_tgt:
        save("train", train_src, train_tgt)
    if val_src and val_tgt:
        save("val", val_src, val_tgt)
    if test_src and test_tgt:
        save("test", test_src, test_tgt)


In [17]:
save_bilingual_splits(
    prefix="UN",
    src_lang="en",
    tgt_lang="ru",
    train_src=train_en, train_tgt=train_ru,
    val_src=val_en, val_tgt=val_ru,
    test_src=test_en, test_tgt=test_ru
)

Saved UN.en-ru.train.en (80,000 lines)
Saved UN.en-ru.train.ru (80,000 lines)
Saved UN.en-ru.val.en (10,000 lines)
Saved UN.en-ru.val.ru (10,000 lines)
Saved UN.en-ru.test.en (10,000 lines)
Saved UN.en-ru.test.ru (10,000 lines)


## Corpus preparation

In [18]:
!git clone https://github.com/ymoslem/MT-Preparation.git

fatal: destination path 'MT-Preparation' already exists and is not an empty directory.


In [19]:
%cd MT-Preparation

/content/MT-Preparation


In [20]:
!pip3 install --user -r requirements.txt

In [21]:
%cd ..

/content


In [22]:
!ls

drive					train.en-de.en-filtered.en
model_cache				train.en-de.en-filtered.en.semantic.en
MT-Preparation				UN.en-ru.test.en
sample_data				UN.en-ru.test.ru
train.en-de.de				UN.en-ru.train.en
train.en-de.de-filtered.de		UN.en-ru.train.ru
train.en-de.de-filtered.de.semantic.de	UN.en-ru.val.en
train.en-de.en				UN.en-ru.val.ru


In [23]:
!python3 MT-Preparation/filtering/filter.py UN.en-ru.train.en UN.en-ru.train.ru en ru

Dataframe shape (rows, columns): (80000, 2)
--- Rows with Empty Cells Deleted	--> Rows: 80000
--- Duplicates Deleted			--> Rows: 79972
--- Source-Copied Rows Deleted		--> Rows: 79971
--- Too Long Source/Target Deleted	--> Rows: 39271
--- Too Short Source/Target Deleted	--> Rows: 39242
--- HTML Removed			--> Rows: 39242
--- Rows will remain true-cased		--> Rows: 39242
--- Rows with Empty Cells Deleted	--> Rows: 39242
--- Rows Shuffled			--> Rows: 39242
--- Source Saved: UN.en-ru.train.en-filtered.en
--- Target Saved: UN.en-ru.train.ru-filtered.ru


In [24]:
!python3 MT-Preparation/filtering/filter.py UN.en-ru.val.en UN.en-ru.val.ru en ru

Dataframe shape (rows, columns): (10000, 2)
--- Rows with Empty Cells Deleted	--> Rows: 10000
--- Duplicates Deleted			--> Rows: 9998
--- Source-Copied Rows Deleted		--> Rows: 9998
--- Too Long Source/Target Deleted	--> Rows: 4739
--- Too Short Source/Target Deleted	--> Rows: 4738
--- HTML Removed			--> Rows: 4738
--- Rows will remain true-cased		--> Rows: 4738
--- Rows with Empty Cells Deleted	--> Rows: 4738
--- Rows Shuffled			--> Rows: 4738
--- Source Saved: UN.en-ru.val.en-filtered.en
--- Target Saved: UN.en-ru.val.ru-filtered.ru


In [25]:
!python3 MT-Preparation/filtering/filter.py UN.en-ru.test.en UN.en-ru.test.ru en ru

Dataframe shape (rows, columns): (10000, 2)
--- Rows with Empty Cells Deleted	--> Rows: 10000
--- Duplicates Deleted			--> Rows: 10000
--- Source-Copied Rows Deleted		--> Rows: 10000
--- Too Long Source/Target Deleted	--> Rows: 4686
--- Too Short Source/Target Deleted	--> Rows: 4685
--- HTML Removed			--> Rows: 4685
--- Rows will remain true-cased		--> Rows: 4685
--- Rows with Empty Cells Deleted	--> Rows: 4685
--- Rows Shuffled			--> Rows: 4685
--- Source Saved: UN.en-ru.test.en-filtered.en
--- Target Saved: UN.en-ru.test.ru-filtered.ru


In [26]:
!python3 MT-Preparation/filtering/semantic_filter.py UN.en-ru.train.en-filtered.en UN.en-ru.train.ru-filtered.ru en ru 5000 #batch 0.45 #treshhold

2025-10-28 09:53:45.412369: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761645225.432719    7486 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761645225.438694    7486 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761645225.455395    7486 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761645225.455419    7486 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761645225.455425    7486 computation_placer.cc:177] computation placer alr

In [27]:
!python3 MT-Preparation/filtering/semantic_filter.py UN.en-ru.val.en-filtered.en UN.en-ru.val.ru-filtered.ru en ru 5000 0.45

2025-10-28 09:53:57.661961: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761645237.681360    7551 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761645237.687321    7551 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761645237.702182    7551 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761645237.702211    7551 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761645237.702215    7551 computation_placer.cc:177] computation placer alr

In [28]:
!python3 MT-Preparation/filtering/semantic_filter.py UN.en-ru.test.en-filtered.en UN.en-ru.test.ru-filtered.ru en ru 5000 0.45

2025-10-28 09:54:50.695503: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761645290.714823    7796 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761645290.720801    7796 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761645290.735449    7796 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761645290.735476    7796 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761645290.735480    7796 computation_placer.cc:177] computation placer alr

In [29]:
!wc -l *

wc: drive: Is a directory
        0 drive
wc: model_cache: Is a directory
        0 model_cache
wc: MT-Preparation: Is a directory
        0 MT-Preparation
wc: sample_data: Is a directory
        0 sample_data
    50000 train.en-de.de
    48315 train.en-de.de-filtered.de
    47614 train.en-de.de-filtered.de.semantic.de
    50000 train.en-de.en
    48315 train.en-de.en-filtered.en
    47614 train.en-de.en-filtered.en.semantic.en
    10000 UN.en-ru.test.en
     4685 UN.en-ru.test.en-filtered.en
        2 UN.en-ru.test.en-filtered.en.semantic.en
    10000 UN.en-ru.test.ru
     4685 UN.en-ru.test.ru-filtered.ru
        2 UN.en-ru.test.ru-filtered.ru.semantic.ru
    80000 UN.en-ru.train.en
    39242 UN.en-ru.train.en-filtered.en
    80000 UN.en-ru.train.ru
    39242 UN.en-ru.train.ru-filtered.ru
    10000 UN.en-ru.val.en
     4738 UN.en-ru.val.en-filtered.en
        7 UN.en-ru.val.en-filtered.en.semantic.en
    10000 UN.en-ru.val.ru
     4738 UN.en-ru.val.ru-filtered.ru
        7 UN.en-ru.v